In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/sf-crime/train.csv.zip')
train

In [ ]:
# 정답컬럼의 값분포 확인 --> 다중분류문제!
train['Category'].value_counts()

In [ ]:
test = pd.read_csv('/kaggle/input/sf-crime/test.csv.zip')
test

In [ ]:
# 데이터가 너무 많으므로 전처리!
all_data = pd.concat([train,test])
all_data

In [ ]:
print(train.columns)
print(test.columns)

In [ ]:
all_data_2 = all_data.drop(columns = ['Category','Dates','Descript','Resolution'])
all_data_2

### address 주소 : 디테일하게 쪼개서 모델이 비슷한것들을 학습할수 있게 해줘야함!
### 텍스트 마이닝

In [ ]:
# Address의 빈도수 체크 --> 지역이름을 컬럼으로 만들어준다(one-hot-encoding과 유사)
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
text = cv.fit_transform(all_data_2['Address'])
text
# 1762311 * 2192  : 로우 개수(all_data_2) * 컬럼개수(지역별로 단어를 쪼개서 컬럼화)

In [ ]:
print(text[0])

In [ ]:
pd.options.display.max_columns=2200
pd.DataFrame(text[0].todense())
# laguna / oak / st를 나눠서 
# 1495/1900/1165 컬럼에 1,2,1 개씩 들어가 있음

In [ ]:
# 컬럼수 너무 많음 --> 차원축소(고차원 벡터 정보를 저차원으로 변환)
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=5)
text_svd = svd.fit_transform(text)
text_svd
# 2200개의 정보를 5개의 성분으로 축소

In [ ]:
# text_svd컬럼을 all_data_2에 추가
for i in range(5):
    all_data_2[f'text_svd_{i}'] = text_svd[:,i]
all_data_2
# Address 텍스트 마이닝 관련 정보가 5개로 축소

In [ ]:
# 문자형 컬럼만 찾기
c = all_data_2.columns[all_data_2.dtypes == object]
c

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

for i in c:
    all_data_2[i] = le.fit_transform(list(all_data_2[i]))
all_data_2

In [ ]:
train_2 = all_data_2[:len(train)]
test_2 = all_data_2[len(train):]

In [ ]:
# 여기에 RandomForest 쓰면? --> 이 데이터셋은 막히는 경우가 많음
# ==> 대용량 데이터라 막히는 경우가 생김, 점수가 낮게 나옴

In [ ]:
# boosting 모델 중 catboost 사용
# 너무 오래 걸리므로 어떻게 하면 점수하락을 최소화하면서 빨리 연산하는지 알아보자!
# --> boosting 모델은 연산시 미분하면서 최적화 --> 사칙연산 작업을 고급인력인 cpu혼자 도맡음
# 사칙연산을 cpu말고 조금 저퀄인력인 gpu에게 맡기자 --> 오른쪽 settings에서 Accelerator : GPU로 바꿈!
from catboost import CatBoostClassifier
cb = CatBoostClassifier(task_type='GPU',)
cb.fit(train_2,train['Category'])

# CatBoost 모델
* Tree 계열 모델(rf,boosting) : Decision Tree를 만들어 여러개의 나무들을 학습시킨 후 학습결과를 종합해서 최종결과 도출
* RandomForest --> Bagging 기법 중 하나
* CatBoost, xgboost, lgbboost --> Boosting 기법의 하나
* 1) Bagging : rf() 는 어느정도는 점수가 잘 나오나 최상위 점수까진 어려움...
* 나무들이 학습하는 방식이 조금씩 다르며, 각 나무들이 독립적이라 서로 모델점수 향상에 도움 X --> 성능이 어느 정도까지만 잘 나오고 그 이상은 안나옴!
* 2) Boosting : 나무들이 순서대로 만들어지면서 1번나무 학습법(오답정리)을 2번나무가 학습가능--> 1번 나무에서 잘못 학습한 부분을 2번나무에서 수정
* 2번, 3번..... 순번이 올라갈수록 성능이 점점 향상됨
* 리스크 : 모델 성능이 잘 나오는게 train 데이터셋을 더 세세하게 학습한다 --> 복잡도가 증가하면서 오버피팅의 위험성 up
* --> hyperparameter tuning 등을 통해서 오버피팅을 어느정도 방지는 가능

In [ ]:
# all_data 데이터셋에서부서 날짜 컬럼추가
all_data['Dates'] = pd.to_datetime(all_data['Dates'])
all_data['year'] = all_data['Dates'].dt.year
all_data['month'] = all_data['Dates'].dt.month
all_data['week'] = all_data['Dates'].dt.week
all_data['day'] = all_data['Dates'].dt.day
all_data['hour'] = all_data['Dates'].dt.hour
all_data['minute'] = all_data['Dates'].dt.minute
# 왜 minute 추가?
# --> 1) 도메인 지식 : 특정 분에 많이 발생하는 범죄가 있을수 있음 ex) 사이버 범죄
# --> 2) 데이터 수집과정에서 다양한 분야의 담당관이 기록하다보니
#     1분/5분 단위로 기록할 수 있음 --> minute이 y값에 영향을 미칠수 있음!

# 범죄가 시간의 흐름에 따라서 달라진게 있을것인데...
# 과거에 비해 얼마나 달라졌는지 알수 있는 컬럼을 만들어야함!
all_data['time'] = (all_data['Dates'].dt.date-all_data['Dates'].dt.date.min()).dt.days
# dt.date() --> 연/월/일 정보가 나옴(사칙연산 가능)
# 최초 시점에서부터 데이터가 새로 기록되기까지 얼마나 지났는지 알 수 있음!
# ().dt.days : days 빼고 숫자만 가져오기 위함!
all_data

In [ ]:
all_data_2 = all_data.drop(columns = ['Category','Dates','Descript','Resolution','Id'])
all_data_2

In [ ]:
c = all_data_2.columns[all_data_2.dtypes == object]
c

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

for i in c :
    all_data_2[i] = le.fit_transform(list(all_data_2[i]))
all_data_2

In [ ]:
train_2 = all_data_2[:len(train)]
test_2 = all_data_2[len(train):]

In [ ]:
# train_test_split() : 제출하지 않고도 모델점수 확인
from sklearn.model_selection import train_test_split
X_train,X_valid,y_train,y_valid = train_test_split(train_2,train['Category'],test_size=0.2,
                                                  stratify=train['Category'],
                                                  random_state=42)
# test_size = validation 데이터셋 사이즈
# random_state = valid : 데이터셋을 명확히 설정하기 위함
#                   --> 시행시 모델 점수가 좋아진게 운이 아니란 것 증명 위함!
# stratify : train데이터셋과 valid데이터셋의 정답컬럼 비율을 일정하게 하기 위함

In [ ]:
'''
# grid/random search 작용(시간이 너무 오래 걸려 다음 기회에...)
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostClassifier
param_grid = {'iterations' : [500,1000,2000,3000,4000,5000,10000],
              'depth' : [4,5,6,7,8],
              'loss_function' : ['MultiClass'],
             'random_seed' : [42]}
cb = CatBoostClassifier(task_type='GPU')
grid_search = GridSearchCV(estimator=cb,param_grid=param_grid,cv=5,n_jobs=-1)
grid_search.fit(X_train,y_train)
'''

In [ ]:
'''
print(grid_search.best_params_)
print(grid_search.best_score_)
'''

In [ ]:
# 평가 데이터셋 적용한 모델링
from catboost import CatBoostClassifier
cb = CatBoostClassifier(task_type='GPU',max_depth=7,verbose=100)
cb.fit(X_train,y_train,eval_set=(X_valid,y_valid))
# 평가 데이터셋의 데이터들은 처음보는 것이므로 test 데이터셋을 마주할때의 상황과 동일!
# 중간에 있는 test점수 --> leaderboard점수!
# max_depth : 하나의 나무에서 얼마나 많이 질문을 던질까? --> 디폴트 : 6

In [ ]:
result = cb.predict_proba(test_2)
result

In [ ]:
pd.Series(cb.feature_importances_,index=train_2.columns)

In [ ]:
pd.Series(cb.feature_importances_,index=train_2.columns).sort_values(ascending=False)

In [ ]:
cb.classes_

In [ ]:
sub = pd.read_csv('/kaggle/input/sf-crime/sampleSubmission.csv.zip')
sub

In [ ]:
sub.iloc[:,1:] = result
sub

In [ ]:
sub.to_csv('test1.csv',index=False)

In [ ]:
# 제출하면서 점수향상 체크하지 말고, 묘듈 내부에서 점수를 올려보자!
# 실제 대회에서는 5번 제출 제한 + 리더보드 존재 X

In [ ]:
from xgboost import XGBClassifier
xgb = XGBClassifier(learning_rate=0.1,tree_method='gpu_hist',n_estimators=10000,
                   colsample_bytree=0.75,subsample=0.75,max_depth=7)
xgb.fit(X_train,y_train,eval_set=[(X_valid,y_valid)],verbose=10,early_stopping_rounds=20)
#xgb.fit(train_2,train['Category'],eval_set=[(X_valid,y_valid)],verbose=10,early_stopping_rounds=20)
result = xgb.predict_proba(test_2)
result

In [ ]:
sub = pd.read_csv('/kaggle/input/sf-crime/sampleSubmission.csv.zip')
sub.iloc[:,1:] = result
sub.to_csv('xgb_test.csv',index=False)